In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import time
from key import id, password

prefs = {
        "download.default_directory": "C:\\Users\\MIRAI\\programs\\syllabus\\pdf\\",
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True,
        }

ID = id
PASS = password

ImportError: cannot import name 'id' from 'key' (c:\programs\PBL_stock\key.py)

In [8]:
options = Options()

options.add_experimental_option("detach", True)

options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

driver.get("https://webcit-5.edu.metro-cit.ac.jp/users/sign_in")

driver.find_element(By.NAME, "user[name]").send_keys(ID)
driver.find_element(By.NAME, "user[password]").send_keys(PASS)

driver.find_element(By.NAME, "commit").click()

In [5]:
time.sleep(1)

driver.execute_script("window.open('https://webcit-5.edu.metro-cit.ac.jp/gnumbers/33494/last_hyoukas');")

# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)

In [6]:
rows = driver.find_elements(By.CSS_SELECTOR, "table.bordered tr")

for row in rows:
    cells = row.find_elements(By.CSS_SELECTOR, "td")
    if cells:
        last_cell = cells[-1]
        if (last_cell.text != "PDFファイルが作成させていません。"):
            # print(last_cell.text)
            last_cell.click()

In [36]:
import os
from PyPDF2 import PdfReader, PdfWriter

def merge_pdfs(directory, output_filename):
    # PdfWriter オブジェクトの初期化
    writer = PdfWriter()

    # 指定されたディレクトリ内のファイルをループする
    for item in sorted(os.listdir(directory)):
        if item.endswith('.pdf'):
            # PDFファイルを開く
            filepath = os.path.join(directory, item)
            reader = PdfReader(filepath)

            # 各ページを書き込む
            for page in reader.pages:
                writer.add_page(page)

    # 結合したPDFを保存
    with open(output_filename, 'wb') as f:
        writer.write(f)

# 使用例
directory = './pdf'
output_filename = './output.pdf'
merge_pdfs(directory, output_filename)

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
import requests
from bs4 import BeautifulSoup
from key import sbi_id, sbi_pass

def remove_symbol_from_json(symbol):
    with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
        data = json.load(file)

    for category in data:
        data[category] = [company for company in data[category] if company["symbol"] != symbol]

    with open('./scrape_data/stock_name.json', 'w', encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

driver.get("https://www.sbisec.co.jp/ETGate")

id_input = driver.find_element(By.NAME, "user_id")
id_input.send_keys(sbi_id)

pass_input = driver.find_element(By.NAME, "user_password")
pass_input.send_keys(sbi_pass)

driver.find_element(By.NAME, "ACT_login").click()

print("Please input Enter")
input("")

driver.get("https://global.sbisec.co.jp/home")
time.sleep(0.3)

driver.find_element(By.XPATH, r'//*[@id="root"]/main/article/div[1]/div[2]/a[2]').click()

# JSONファイルの読み込み
with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

# 全てのシンボルをプリント
for category in data:
    for company in data[category]:
        stock_code = company["symbol"]
        stock_name = company["company_name"]
        print(stock_code, ':', stock_name)

        try:
            driver.get(f"https://global.sbisec.co.jp/invest/us/stock/{stock_code}?resource=news&searchType=include")
            time.sleep(0.3)

            iframe = driver.page_source
            soup = BeautifulSoup(iframe, "html.parser")
            url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"
            iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

            token = iframe_urls[0].split("token=")[1]
            url_head = f"https://graph.sbisec.co.jp/sbinews/srvdetail?symbol={stock_code}&token={token}"
            print(url_head)

            res_head = requests.get(url_head)
            data_head = res_head.json()

            len_head = len(data_head['data'])
            time.sleep(1)

        except Exception as e:
            print(f"Error processing {stock_code}: {e}")
            remove_symbol_from_json(stock_code)
            print(f"Removed {stock_code} from stock_name.json")


Please input Enter
A : アジレント・テクノロジー Agilent Technologies, Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=A&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BABAB8088A49AB6CCFC7AACEB132F67F8EECD5EB6BCEA2C51CD828C00
AAL : アメリカン・エアラインズ・グループ American Airlines Group Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=AAL&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BABAB8088A49AB6CCFC7AACEB132F67F8EECD5EB6BCEA2C51CD828C00
AAPL : アップル Apple Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=AAPL&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BABAB8088A49AB6CCFC7AACEB132F67F8EECD5EB6BCEA2C51CD828C00
ABBV : アッヴィ AbbVie Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=ABBV&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BAB